# Reranking using SciBERT

First, follow [this Notebook](https://github.com/castorini/anserininotebooks/blob/master/Pyserini%2BSciBERT_on_COVID_19_Demo.ipynb) to reproduce SciBERT, then try `BertForNextSentence` to see if it is better.





In [2]:
# Download Anserini Run on Abstract 
# https://github.com/castorini/anserini/blob/master/docs/experiments-covid.md

!wget -P ../tmp https://www.dropbox.com/s/4zau6ejrkvgn9m7/anserini.covid-r4.abstract.qdel.bm25.txt

--2020-08-11 05:08:50--  https://www.dropbox.com/s/4zau6ejrkvgn9m7/anserini.covid-r4.abstract.qdel.bm25.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4zau6ejrkvgn9m7/anserini.covid-r4.abstract.qdel.bm25.txt [following]
--2020-08-11 05:08:50--  https://www.dropbox.com/s/raw/4zau6ejrkvgn9m7/anserini.covid-r4.abstract.qdel.bm25.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc91d4844cf95e7936fe27411683.dl.dropboxusercontent.com/cd/0/inline/A9Pie6PlFO4Lsk8k9dV6VXe78_Nt5CrSydgN35HsJAM2K-2x9AEyQbE34_wIVzP_UCt_lSmBwNQEdAcB_Ll0PMKM0zHSxFkLE-SIDwa1qCoGB9KaEnqeG_NoqZLF5AJfA3g/file# [following]
--2020-08-11 05:08:50--  https://uc91d4844cf95e7936fe27411683.dl.dropboxusercontent.com/cd/0/inline/A9Pie6PlFO4Lsk8k9dV6VXe78_Nt5C

In [ ]:
import csv 
import torch
from xml.etree import ElementTree
import numpy
from tqdm import tqdm
from transformers import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

PATH_TO_CORD_19_DATA = "../../CORD-19/2020-06-19/"
PATH_TO_ANSERINI_RUN = "../tmp/anserini.covid-r4.abstract.qdel.bm25.txt"
PATH_TO_TOPICS = "../tmp/topics-rnd4.xml"

In [9]:
with open(PATH_TO_ANSERINI_RUN, "r") as f:
    anserini_run = f.readlines()

In [12]:
# get abstract from dataset

def get_abstract(path):
    abstracts = dict()
    with open(path + "metadata.csv") as f_in:
        reader = csv.DictReader(f_in)
        for row in reader:
            cord_uid = row['cord_uid']
            abstract = row['abstract']
            abstracts[cord_uid] = abstract
            
    return abstracts


abstracts_dict = get_abstract(PATH_TO_CORD_19_DATA)



In [19]:
# remove abstract that we won't rerank
TOP_K = 200

temp = dict()
count = 0
qid_prev = None

for line in anserini_run:
    qid, _, uid, _, _, _ = line.strip().split()
    if qid != qid_prev:
        count = 0
        qid_prev = qid
    if count > TOP_K:
        continue    
    
    
    temp[uid] = abstracts_dict[uid]
    count += 1

abstracts_dict = temp

In [17]:
tokenizer = AutoTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed', do_lower_case=False)
model = AutoModel.from_pretrained('monologg/biobert_v1.1_pubmed')

In [37]:
CHUNK_SIZE = 510

# Waring can be ignored. We won't pass the entire sequence to the model anyway.

def extract_scibert(text, tokenizer, model):
    text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

    n_chunks = int(numpy.ceil(float(text_ids.size(1))/CHUNK_SIZE))
    states = []
    
    for ci in range(n_chunks):
        text_ids_ = text_ids[0, 1+ci*CHUNK_SIZE:1+(ci+1)*CHUNK_SIZE]            
        text_ids_ = torch.cat([text_ids[0, 0].unsqueeze(0), text_ids_])
        if text_ids[0, -1] != text_ids[0, -1]:
            text_ids_ = torch.cat([text_ids_, text_ids[0,-1].unsqueeze(0)])
        
        with torch.no_grad():
            state = model(text_ids_.unsqueeze(0))[0]
            state = state[:, 1:-1, :]
        states.append(state)

    state = torch.cat(states, axis=1)
    return text_ids, text_words, state[0]

In [ ]:
encoded_abstract = dict()

for k, text in abstracts_dict.items():
    encoded_abstract[text] = extract_scibert(text, tokenizer, model)

Token indices sequence length is longer than the specified maximum sequence length for this model (826 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (648 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (935 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (693 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (585 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2758 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (701 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [ ]:
def read_topics(path_to_topic):
    tree = ElementTree.parse(path_to_topic)
    topics = list()
    for topic in tree.getroot():
        d = dict()
        d["number"] = topic.attrib["number"]
        for field in topic:
            d[field.tag] = field.text
        topics.append(d)

    return topics

topics = read_topics(PATH_TO_TOPICS)
encoded_topics = dict()

for topic in topics:
    encoded_topics[topic] = extract_scibert(text, tokenizer, model)